## XP 

In [ ]:
import sys
from pathlib import Path

# Ajoute le dossier racine du projet dans le PYTHONPATH
project_root = Path("/home/tibia/Projet_Hemorragie/Seg_hemorragie/Multitask_model")
sys.path.append(str(project_root))

import data.dataset as dataset
import data.transform as T_mtsk

# Import depuis models/
from models.architecture import BasicUNetWithClassification
from models.lightning_module import MultiTaskHemorrhageModule, MultiTaskHemorrhageModule_homeo, MultiTaskHemorrhageModule_gradnorm

import utils


/home/tibia/Projet_Hemorragie/hemorragie-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import wandb
import torch


def load():    
    train_data=dataset.get_balanced_multitask_dataset('train')
    val_data=dataset.get_balanced_multitask_dataset('val')
    return train_data, val_data

In [ ]:
def load_and_log():
    # 🚀 start a run, with a type to label it and a project it can call home
    with wandb.init(project="artifacts-v0", job_type="load-data_pos") as run:
            
        datasets = load()  # separate code for loading the datasets
        
        names = ["training", "validation"]
            # 🏺 create our Artifact
        raw_data = wandb.Artifact(
            "hemorrhage-multitask-raw",
            type="dataset",
            description="Dataset multitâche Hémorragie (positifs uniquement)"
        )

        for name, data in zip(names, datasets):
                # 🐣 Store a new file in the artifact, and write something into its contents.
                with raw_data.new_file(name + ".pt", mode="wb") as file:
                    x, y = data.tensors
                    torch.save((x, y), file)

            # ✍️ Save the artifact to W&B.
        run.log_artifact(raw_data)

load_and_log()